Imports

In [1]:
import numpy as np
import pandas as pd
import obspy
from obspy import read
from scipy.signal import butter, filtfilt, wiener

Load and Preprocess the Data

In [ ]:
def load_data(file_path, file_type='csv'):
    if file_type == 'csv':
        data = pd.read_csv(file_path)
        x = data['data'].values
        arrival_time = data['arrival_time'].values
    elif file_type == 'mseed':
        st = read(file_path)
        x = st[0].data
        arrival_time = st[0].stats.starttime.timestamp
    else:
        raise ValueError("Unsupported file type")
    
    y = np.zeros_like(x)
    y[int(arrival_time)] = 1
    return x, y

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def preprocess_data(x, fs=100):
    x = bandpass_filter(x, 1.0, 10.0, fs)
    x = wiener(x)
    return x

Define the STA/LTA Algorithm

In [ ]:
def recursive_sta_lta(data, nsta, nlta):
    sta = np.cumsum(data ** 2)
    lta = np.cumsum(data ** 2)
    sta[nsta:] = sta[nsta:] - sta[:-nsta]
    lta[nlta:] = lta[nlta:] - lta[:-nlta]
    sta /= nsta
    lta /= nlta
    sta_lta = sta / lta
    return sta_lta

def carl_sta_lta(data, nsta, nlta):
    sta = np.cumsum(data ** 2)
    lta = np.cumsum(data ** 2)
    sta[nsta:] = sta[nsta:] - sta[:-nsta]
    lta[nlta:] = lta[nlta:] - lta[:-nlta]
    sta /= nsta
    lta /= nlta
    sta_lta = sta / lta
    return sta_lta

def short_time_fourier_transform(data, window_size, step_size):
    stft = []
    for i in range(0, len(data) - window_size, step_size):
        window = data[i:i + window_size]
        stft.append(np.fft.fft(window))
    return np.array(stft)

Define the 1D CNN Model

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input

def create_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        Conv1D(16, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Conv1D(32, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

Adjust the Loss Function

In [ ]:
def custom_loss(y_true, y_pred):
    loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    weight = tf.where(tf.equal(y_true, 1), 10.0, 1.0)
    return loss * weight

Train the Model

In [ ]:
def train_model(model, x_train, y_train, epochs=50, batch_size=32):
    model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

 Evaluate the Model

In [ ]:
def evaluate_model(model, x_test, y_test):
    loss, accuracy = model.evaluate(x_test, y_test)
    print(f'Test Accuracy: {accuracy:.4f}')

Main Script

In [ ]:
if __name__ == "__main__":
    # Load and preprocess data
    x, y = load_data('data.csv', file_type='csv')
    x = preprocess_data(x)
    
    # Reshape data for CNN
    x = x.reshape(-1, len(x), 1)
    y = y.reshape(-1, len(y), 1)
    
    # Create and train the model
    model = create_model(input_shape=(x.shape[1], 1))
    train_model(model, x, y, epochs=50, batch_size=32)
    
    # Evaluate the model
    evaluate_model(model, x, y)